<a href="https://colab.research.google.com/github/AnyelaFransheska/proyecto-de-grafo-fisica/blob/main/ColabproyectodegradoAFMV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Librarys
import json
import pandas as pd

import requests

import librosa
import librosa.display as dsp
import io
from scipy import signal
import numpy as np
import numpy.fft as fft
import math


import seaborn as sns
import matplotlib.pyplot as plt
import folium

# import nolds  #libreria para calcular la entropia
import numpy as np
import scipy.fftpack as fourier

import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_io as tfio
from IPython.display import Audio

import random
import sklearn
from sklearn import *

sns.set_theme()

In [ ]:
#  Funciones-> Graficar
def grafCount(df,message,name):
  plt.figure(figsize=(16, 6))
  ax = sns.countplot(x=df, palette="hls",order = df.unique().sort())
  # ax.add_artist(ab)

  plt.title(f"Audio files: {message} ", fontsize=16)
  plt.xticks(rotation=90, fontsize=13)
  plt.yticks(fontsize=13)
  plt.ylabel("Frequency", fontsize=14)
  plt.xlabel("");
  plt.savefig(name+"_grafcount"+".png") # pegar en cada funcion para guardar las img

def colorhex():
  r = lambda: random.randint(0,255)
  return '#%02X%02X%02X' % (r(),r(),r())

def markMap(lng,lat,name,m):
  tooltip = "lat:{},lng:{}".format(lat,lng)
  folium.Marker(
    [lng,lat], popup=name, tooltip=tooltip).add_to(m)
    #plt.savefig(name+"_Markmap"+".png") # pegar en cada funcion para guardar las img

# Función que normaliza los datos de sonido
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)



#  Funciones

def linearFreq(data,sample_rate ,name):
  d = librosa.stft(data) # DFT
  D = librosa.amplitude_to_db(np.abs(d),ref=np.max)
  times = librosa.times_like(D)

  fig,ax = plt.subplots(2,1,sharex=True,figsize=(10,10))
  img = dsp.specshow(D, y_axis='linear', x_axis='s',sr=sample_rate,ax=ax[0])
  ax[0].set(title=f'Linear frequency power spectrogram {name}')
  ax[0].label_outer()

  dsp.specshow(D,y_axis='log',x_axis='s',sr=sample_rate,ax=ax[1])
  # F_fund = freqF0(audio)
  # ff_ = np.ones(D.T.shape)*F_fund
  ax[1].set(title='Log frequency power spectrogram')
  # ax[1].plot(times, ff_[:len(times)], label=f'Freq fundamental {round(F_fund[0],2)} Hz', color='w')
  ax[1].label_outer()
  fig.colorbar(img, ax=ax, format='%+2.f dB')
  plt.savefig(name+"_Freq"+".png") # pegar en cada funcion para guardar las img
  plt.show()


def freqF0(audio,sample_rate):
  gk = fourier.fft(audio)                             # Calculamos la FFT
  M_gk = abs(gk)                                  # Calculamos la Magnitud de la FFT
  F = sample_rate*np.arange(0, len(audio))/len(audio)              # Definimos el Vector de Frecuencias
  M_gk= M_gk[:int(len(M_gk)/2)]
  F= F[:int(len(F)/2)]
  Posm = np.where(M_gk == np.max(M_gk))           # Encontramos la posición para la cual la Magnitud de FFT es máxima
  F_fund = F[Posm]
  return F_fund

def centSpect(data,sample_rate,name):

  F_fund = freqF0(audio,sample_rate)
  cent = librosa.feature.spectral_centroid(y=audio, sr=sample_rate)

  S, phase = librosa.magphase(librosa.stft(y=audio));
  librosa.feature.spectral_centroid(S=S);
  times = librosa.times_like(cent)
  fig, ax = plt.subplots(figsize=(10,10))
  img =librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),
                            y_axis='log', x_axis='time', ax=ax)
  ax.plot(times, cent.T, label='Spectral centroid', color='w')
  ff_ = np.ones(cent.T.shape)*F_fund
  ax.plot(times, ff_, label=f'Freq fundamental {round(F_fund[0],2)} Hz', color='w')
  ax.legend(loc='upper right')
  ax.set(title=f'log Power spectrogram: {name}')
  fig.colorbar(img, ax=ax, format='%+2.f dB')
  plt.savefig(name+"_Centroide"+".png") # pegar en cada funcion para guardar las img

  # plt.colorbar()
  plt.show()

#


def periodograma(audio,sample_rate,name):
  f, Pxx_den = signal.periodogram(audio, sample_rate)
  plt.semilogy(f, Pxx_den)
  plt.ylim([1e-7, 1e2])
  # plt.plot(f,Pxx_den)
  plt.xlabel('frequency [Hz]')
  plt.ylabel('PSD [V**2/Hz]')
  plt.savefig(name+"_Periodograma"+".png") # pegar en cada funcion para guardar las img
  plt.show()

def armonicos(audio,sample_rate,name):
  n = audio.size
  p = np.fft.rfft(audio)
    # mag = np.linalg.norm(p)
  mag = np.sqrt(p.real**2+p.imag**2)
  maG = mag
  maG = maG[0:math.ceil(n/2)]
  freq =np.arange(0,maG.size,1)*(sample_rate/n)
  # freq = sample_rate*np.arange(0, len(audio))/len(audio)


  # freq.shape[0]/2
  freq.shape
  # plt.title("Armonicos")
  plt.plot(freq,maG)
  Posm = np.where(maG == np.max(maG))           # Encontramos la posición para la cual la Magnitud de FFT es máxima
  F_fund = freq[Posm]
  print(F_fund)
  plt.plot(F_fund,maG[Posm],'k*',label=f'F0: {round(F_fund[0],2)} Hz')
  # plt.xticks(np.arange(np.min(freq),np.max(freq)+1))
  # plt.show()
  plt.xlabel('Frecuencia (Hz)', fontsize='14')
  plt.ylabel('Amplitud FFT', fontsize='14')
  plt.savefig(name+"_Armonicos"+".png") # pegar en cada funcion para guardar las img
  plt.legend()
  plt.show()



def wave_spect_bird(audio,sample_rate,name):
  librosa.display.waveshow(audio, sr=sample_rate)
  plt.title('Forma de onda')
  plt.xlabel('Tiempo (segundos)')
  plt.ylabel('Amplitud')
  plt.savefig(name+"_Wave"+".png") # pegar en cada funcion para guardar las img
  plt.show()

  # Visualizar espectrograma
  D = librosa.amplitude_to_db(abs(librosa.stft(audio)), ref=np.max)
  librosa.amplitude_to_db(S,  ref=np.max )
  # plt.figure(figsize=(12, 4))
  librosa.display.specshow(D, y_axis='log', x_axis='time')
  plt.colorbar(format='%+2.0f dB')
  plt.title('Espectrograma')
  plt.xlabel('Tiempo (segundos)')
  plt.ylabel('Frecuencia (Hz)')
  plt.savefig(name+"_Espectograna"+".png") # pegar en cada funcion para guardar las img
  plt.show()

def chromagramBird(audio,sample_rate,name):

  # Un chromagram es una representación visual de la energía espectral de una señal de audio
  # en función del tiempo y la altura de la nota musical.
  C = np.abs(librosa.stft(audio))
  chroma = librosa.feature.chroma_stft(S=C, sr=sample_rate)

  fig, ax = plt.subplots(figsize=(10,6))
  img = librosa.display.specshow(chroma, y_axis='chroma', x_axis='s', ax=ax)
  fig.colorbar(img, ax=ax)
  ax.set(title='Chromagram')
  plt.savefig(name+"_Chromagram"+".png") # pegar en cada funcion para guardar las img
  plt.show()


def tempo_bird(audio,sample_rate):
  oenv = librosa.onset.onset_strength(y=audio, sr=sample_rate)
  # Estimate the global tempo for display purposes
  tempo = librosa.beat.tempo(onset_envelope=oenv, sr=sample_rate)[0]
  return tempo


def tempogramBird(audio,sample_rate,name):
  oenv = librosa.onset.onset_strength(y=audio, sr=sample_rate)
  tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sample_rate)

  # Compute global onset autocorrelation
  ac_global = librosa.autocorrelate(oenv, max_size=tempogram.shape[0])
  ac_global = librosa.util.normalize(ac_global)

  # Estimate the global tempo for display purposes
  tempo = librosa.beat.tempo(onset_envelope=oenv, sr=sample_rate)[0]

  fig, ax = plt.subplots(nrows=2, figsize=(10, 8))
  times = librosa.times_like(oenv, sr=sample_rate)
  ax[0].plot(times, oenv, label='Onset strength')
  ax[0].label_outer()
  ax[0].legend(frameon=True)
  librosa.display.specshow(tempogram, sr=sample_rate,x_axis='s',
      y_axis='tempo', cmap='magma',ax=ax[1])
  ax[1].axhline(tempo, color='g', linestyle='--', alpha=1,
              label='Estimated tempo={:g}'.format(tempo))
  ax[1].legend(loc='upper right')
  ax[1].set(title='Tempogram')
  plt.savefig(name+"_Audioreconstruido"+".png") # pegar en cada funcion para guardar las img

  plt.show()

  return tempo


# Spectral RollOff Vector
def spectralRolloff(audio, sample_rate,name):
  spectral_rolloff = librosa.feature.spectral_rolloff(y = audio, sr=sample_rate)[0]

  # Computing the time variable for visualization
  # frames = range(len(spectral_rolloff))
  frames = range(len(spectral_rolloff))
  # Converts frame counts to time (seconds)
  t = librosa.frames_to_time(frames)
  norm =  normalize(spectral_rolloff)
  n= len(audio)
  print(len(spectral_rolloff))
  print(n)

  # The plot
  plt.figure(figsize = (16, 6))
  librosa.display.waveshow(y=audio, sr=sample_rate, alpha=0.4, color = '#A300F9', lw=3)
  plt.plot(t[:n],norm[:n] , color='#FFB100', lw=3)
  plt.legend(["Spectral Rolloff", "Wave"])
  plt.title(f"Spectral Rolloff: {name}", fontsize=16);
  plt.savefig(name+"_SpectralRolloff"+".png") # pegar en cada funcion para guardar las img

def melSpectogramBird(audio,sample_rate,name):
  # Create the Mel Spectrograms
  n_fft = 2048 # FFT window size
  hop_length = 512 # number audio of frames between STFT columns (looks like a good default)
  S_amered = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
  S_DB_amered = librosa.amplitude_to_db(S_amered, ref=np.max)
  librosa.display.specshow(S_DB_amered, sr = sample_rate, hop_length = hop_length, x_axis = 'time',
                          y_axis = 'hz')
  plt.title(f"melspectogram: {name}")
  plt.savefig(name+"_Melspectogram"+".png") # pegar en cada funcion para guardar las img
  plt.show()

def rms_bird(audio,name):
  """
  Compute root-mean-square (RMS) value for each frame, either from the audio samples y or from a spectrogram S
  """
  y = audio
  S, phase = librosa.magphase(librosa.stft(y))
  rms = librosa.feature.rms(S=S)
  fig, ax = plt.subplots(nrows=2, sharex=True)
  times = librosa.times_like(rms)
  ax[0].set(title=f"RMS: {name}")
  ax[0].semilogy(times, rms[0], label='RMS Energy')
  ax[0].set(xticks=[])
  ax[0].legend()
  ax[0].label_outer()
  librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),
                          y_axis='log', x_axis='time', ax=ax[1])
  ax[1].set(title='log Power spectrogram')
  S = librosa.magphase(librosa.stft(y, window=np.ones, center=False))[0]
  librosa.feature.rms(S=S)
  plt.savefig(name+"_RMS"+".png") # pegar en cada funcion para guardar las img
  plt.show()


def bandwith_bird(audio,sample_rate,name):
  y = audio
  sr = sample_rate
  spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
  freqs, times, D = librosa.reassigned_spectrogram(y, fill_nan=True)
  librosa.feature.spectral_bandwidth(S=np.abs(D), freq=freqs)
  S, phase = librosa.magphase(librosa.stft(y=y))
  fig, ax = plt.subplots(nrows=2, sharex=True)
  times = librosa.times_like(spec_bw)
  centroid = librosa.feature.spectral_centroid(S=S)
  ax[0].semilogy(times, spec_bw[0], label='Spectral bandwidth')
  ax[0].set(ylabel='Hz', xticks=[], xlim=[times.min(), times.max()])
  ax[0].set(title=f"Spectral bandwoth: {name}")
  ax[0].legend()
  ax[0].label_outer()
  librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),
                          y_axis='log', x_axis='time', ax=ax[1])
  ax[1].set(title='log Power spectrogram')
  ax[1].fill_between(times, np.maximum(0, centroid[0] - spec_bw[0]),
                  np.minimum(centroid[0] + spec_bw[0], sr/2),
                  alpha=0.5, label='Centroid +- bandwidth')
  ax[1].plot(times, centroid[0], label='Spectral centroid', color='w')
  ax[1].legend(loc='lower right')
  plt.savefig(name+"_C+B"+".png") # pegar en cada funcion para guardar las img

def Spectralcontrast_bird(audio,sample_rate,name):
  y, sr = audio, sample_rate
  S = np.abs(librosa.stft(y))
  contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
  fig, ax = plt.subplots(nrows=2, sharex=True)
  img1 = librosa.display.specshow(librosa.amplitude_to_db(S,
                                                  ref=np.max),
                          y_axis='log', x_axis='time', ax=ax[0])
  fig.colorbar(img1, ax=[ax[0]], format='%+2.0f dB')
  img1.set_clim(vmin=0, vmax=1000) #agregado para modificar lo de los db

  ax[0].set(title=f'Power spectrogram: {name}')
  ax[0].label_outer()
  img2 = librosa.display.specshow(contrast, x_axis='time', ax=ax[1])
  fig.colorbar(img2, ax=[ax[1]])
  ax[1].set(ylabel='Frequency bands', title='Spectral contrast')
  plt.savefig(name+"_Spectralcontrast"+".png") # pegar en cada funcion para guardar las img

def spectral_roll_por(audio,sample_rate,name):

  y ,sr = audio, sample_rate

  rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.99)

  # Approximate minimum frequencies with roll_percent=0.01
  rolloff_min = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.01)

  fig, ax = plt.subplots()

  librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),
                          y_axis='log', x_axis='time', ax=ax)
  ax.plot(librosa.times_like(rolloff), rolloff[0], label='Roll-off frequency (0.99)')
  ax.plot(librosa.times_like(rolloff), rolloff_min[0], color='w',
          label='Roll-off frequency (0.01)')
  ax.legend(loc='lower right')
  ax.set(title=f'log Power spectrogram: {name}')
  plt.savefig(name+"_Rolloffrequency"+".png") # pegar en cada funcion para guardar las img


def poly_features_bird(audio,name):
  y = audio

  S = np.abs(librosa.stft(y))
  p0 = librosa.feature.poly_features(S=S, order=0)
  p1 = librosa.feature.poly_features(S=S, order=1)
  p2 = librosa.feature.poly_features(S=S, order=2)
  fig, ax = plt.subplots(nrows=4, sharex=True, figsize=(8, 8))
  times = librosa.times_like(p0)
  ax[0].plot(times, p0[0], label='order=0', alpha=0.8)
  ax[0].plot(times, p1[1], label='order=1', alpha=0.8)
  ax[0].plot(times, p2[2], label='order=2', alpha=0.8)
  ax[0].legend()
  ax[0].label_outer()
  ax[0].set(ylabel='Constant term ')
  ax[0].set(title=f'Poly: {name} ')
  ax[1].plot(times, p1[0], label='order=1', alpha=0.8)
  ax[1].plot(times, p2[1], label='order=2', alpha=0.8)
  ax[1].set(ylabel='Linear term')
  ax[1].label_outer()
  ax[1].legend()
  ax[2].plot(times, p2[0], label='order=2', alpha=0.8)
  ax[2].set(ylabel='Quadratic term')
  ax[2].legend()
  librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),
                          y_axis='log', x_axis='time', ax=ax[3])
  plt.savefig(name+"_polyfeatures"+".png") # pegar en cada funcion para guardar las img

def tonnetz_bird(audio,sample_rate,name):
  y, sr = audio,sample_rate
  y = librosa.effects.harmonic(y)
  tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
  fig, ax = plt.subplots(nrows=2, sharex=True)
  img1 = librosa.display.specshow(tonnetz,
                                  y_axis='tonnetz', x_axis='time', ax=ax[0])
  ax[0].set(title=f'Tonal Centroids (Tonnetz): {name}')
  ax[0].label_outer()
  img2 = librosa.display.specshow(librosa.feature.chroma_cqt(y=y, sr=sr),
                                  y_axis='chroma', x_axis='time', ax=ax[1])
  ax[1].set(title='Chroma')
  fig.colorbar(img1, ax=[ax[0]])
  fig.colorbar(img2, ax=[ax[1]])
  plt.savefig(name+"_Chroma"+".png") # pegar en cada funcion para guardar las img

def harmonic_percep(audio,name):

  y_harm_haiwoo, y_perc_haiwoo = librosa.effects.hpss(audio)

  plt.figure(figsize = (16, 6))
  plt.plot(y_perc_haiwoo, color = '#FFB100')
  plt.plot(y_harm_haiwoo, color = '#A300F9')
  plt.legend(("Perceptrual", "Harmonics"))
  plt.title(f"Harmonics and Perceptrual : {name}", fontsize=16);
  plt.savefig(name+"_Armonicospercep"+".png") # pegar en cada funcion para guardar las img



def decompose_rec(audio,name):
  y=audio
  S = np.abs(librosa.stft(y))

  comps, acts = librosa.decompose.decompose(S, n_components=16)
  T = sklearn.decomposition.MiniBatchDictionaryLearning(n_components=16)
  scomps, sacts = librosa.decompose.decompose(S, transformer=T, sort=True)

  layout = [list(".AAAA"), list("BCCCC"), list(".DDDD")]
  fig, ax = plt.subplot_mosaic(layout, constrained_layout=True)
  librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),
                          y_axis='log', x_axis='time', ax=ax['A'])
  ax['A'].set(title='Input spectrogram')
  ax['A'].label_outer()
  librosa.display.specshow(librosa.amplitude_to_db(comps,
                                                  ref=np.max),
                          y_axis='log', ax=ax['B'])
  ax['B'].set(title='Components')
  ax['B'].label_outer()
  ax['B'].sharey(ax['A'])
  librosa.display.specshow(acts, x_axis='time', ax=ax['C'], cmap='gray_r')
  ax['C'].set(ylabel='Components', title='Activations')
  ax['C'].sharex(ax['A'])
  ax['C'].label_outer()
  S_approx = comps.dot(acts)
  img = librosa.display.specshow(librosa.amplitude_to_db(S_approx,
                                                        ref=np.max),
                                y_axis='log', x_axis='time', ax=ax['D'])
  ax['D'].set(title='Reconstructed spectrogram')
  ax['D'].sharex(ax['A'])
  ax['D'].sharey(ax['A'])
  ax['D'].label_outer()
  fig.colorbar(img, ax=list(ax.values()), format="%+2.f dB")
  plt.savefig(name+"_decompose"+".png") # pegar en cada funcion para guardar las img
  S = librosa.feature.inverse.mel_to_stft(S_approx)



  return librosa.griffinlim(S)


 #plt.savefig(name+"_getdat"+".png") # pegar en cada funcion para guardar las img

# audio, sample_rate = librosa.core.load(io.BytesIO(db), mono=True, sr=df.smp[k], dtype=np.float,res_type='kaiser_fast')
import math
from collections import Counter
def shannon_entropy(audio):

    # Calcula la frecuencia de cada elemento en los datos.
    freq_dict = Counter(audio)
    # Calcula la probabilidad de cada elemento.
    probs = [freq_dict[element] / float(len(audio)) for element in freq_dict]
    # Calcula la entropía de Shannon.
    entropy = -sum([p * math.log(p, 2) for p in probs])
    return entropy



#frecuencias fundamental y armonicos secundarios
def freqF0_and_harmonics(audio, sample_rate, num_harmonics=5):
    import numpy as np
    from numpy import fft

    gk = fft.fft(audio)                                # Calculamos la FFT
    M_gk = abs(gk)                                     # Calculamos la Magnitud de la FFT
    F = sample_rate * np.arange(0, len(audio)) / len(audio)  # Definimos el Vector de Frecuencias
    M_gk = M_gk[:int(len(M_gk) / 2)]                   # Tomamos la mitad del espectro (propiedades de señales reales)
    F = F[:int(len(F) / 2)]

    # Encontramos las posiciones para las cuales la Magnitud de FFT es máxima (fundamental y armonicos)
    pos_max = np.argsort(M_gk)[::-1][:num_harmonics+1]

    freqs = F[pos_max]
    magnitudes = M_gk[pos_max]

    return freqs, magnitudes





def getDat_bird(db,k,nameA):
  audio, sample_rate = librosa.core.load(io.BytesIO(db), mono=True, sr=k,res_type='kaiser_fast')
  # linearFreq(audio,sample_rate,nameA)
  # centSpect(audio,sample_rate,nameA)
  # periodograma(audio,sample_rate,nameA)
  # armonicos(audio,sample_rate,nameA)
  # chromagramBird(audio,sample_rate,nameA)
  # melSpectogramBird(audio,sample_rate,nameA)
  # spectralRolloff(audio,sample_rate,nameA)
  # bandwith_bird(audio,sample_rate,nameA)
  # Spectralcontrast_bird(audio,sample_rate,nameA)
  # poly_features_bird(audio,nameA)
  # tonnetz_bird(audio,sample_rate,nameA)
  # Y_appr= decompose_rec(audio)
  f0 = freqF0(audio,sample_rate)
  # tempoB = tempogramBird(audio,sample_rate,nameA)
  tempoB = tempo_bird(audio,sample_rate)
  #plt.savefig(name+"_tempoB"+".png") # pegar en cada funcion para guardar las img
  # print("Frecuencia de muestreo {} Hz".format(sample_rate))
  # t = audio.size/sample_rate

  # print("Duracion: {:.2f} s".format(t))
  # # librosa.get_duration(audio,sample_rate)
  # print('y:', audio, '\n')
  # print('y shape:', np.shape(audio), '\n')
  # print('Sample Rate (KHz):', sample_rate, '\n')

  # # Verify length of the audio
  # print('Check Len of Audio:', np.shape(audio)[0]/sample_rate)
  Entropia=shannon_entropy(audio)
  Armonicos=freqF0_and_harmonics(audio, sample_rate, num_harmonics=5)
  return f0, tempoB, audio,Entropia,Armonicos #, Y_appr


In [ ]:
# Data processing
import pandas as pd
import numpy as np
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Dataset
from sklearn import datasets
# Dimensionality reduction
from sklearn.decomposition import PCA
# Modeling
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as hier
from sklearn.mixture import GaussianMixture
# Number of clusters
from sklearn.metrics import silhouette_score



def optimalK(data, nrefs=3, maxClusters=15):
    """
    Calculates KMeans optimal K using Gap Statistic from Tibshirani, Walther, Hastie
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for gap_index, k in enumerate(range(1, maxClusters)):
        # Holder for reference dispersion results
        refDisps = np.zeros(nrefs)
        # For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):

            # Create new random reference set
            randomReference = np.random.random_sample(size=data.shape)

            # Fit to it
            km = KMeans(k)
            km.fit(randomReference)

            refDisp = km.inertia_
            refDisps[i] = refDisp
        # Fit cluster to original data and create dispersion
        km = KMeans(k)
        km.fit(data)

        origDisp = km.inertia_
        # Calculate gap statistic
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)
        # Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap

        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap}, ignore_index=True)
    return (gaps.argmax() + 1, resultsdf)  # Plus 1 because index of 0 means 1 cluster is optimal, index 2 = 3 clusters are optimal


# Put features data into a dataframe
df = pd.read_csv(r"bdbirdelevation3.csv")
# Add target to the dataframe

X = df[['Temp','F0','tempo','entropy']]

# Automatically output the number of clusters
k, gapdf = optimalK(X, nrefs=3, maxClusters=11)
print('Optimal k is: ', k)
# Visualization
plt.plot(gapdf.clusterCount, gapdf.gap, linewidth=3)
plt.scatter(gapdf[gapdf.clusterCount == k].clusterCount, gapdf[gapdf.clusterCount == k].gap, s=250, c='r')
plt.grid(True)
plt.xlabel('Cluster Count')
plt.ylabel('Gap Value')
plt.title('Gap Values by Cluster Count')
plt.show()

In [ ]:
#codigo 2 k-means

# IMPORTAR LAS LIBRERIAS
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objs as go
from plotly import tools
from plotly.subplots import make_subplots
import plotly.offline as py
import plotly.graph_objects as go

import plotly.io as pio

!pip install opencv-python
!pip install dash_html_components
!pip install dash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output

#MOSTRAR EN EL NAVEGADOR para visual,
#pio.renderers.default ='google-chrome'

#LEER EL DATASET
df = pd.read_csv(r"bdbirdelevation3.csv")
#TOMA LOS VALORES DE LAS COLUMNAS Y LOS VUELVE UN ARREGLO
x =  df.iloc[:,[1,5,8]].values
#x =  df.iloc[:,[9,6,5,1]].values
#%%
# Este método utiliza los valores de la inercia obtenidos tras aplicar el K-means a diferente número de Clusters,
#siendo la inercia la suma de las distancias al cuadrado de cada objeto del Cluster a su centroide
WCSS = []
#
for i in range(1,10):
    model = KMeans(n_clusters = i,init = 'k-means++')
    model.fit(x)
    WCSS.append(model.inertia_)
fig = plt.figure(figsize = (7,7))
plt.plot(range(1,10),WCSS, linewidth=4, markersize=12,marker='o',color = 'green')
plt.xticks(np.arange(11))
plt.title("Método del Codo ")
plt.xlabel("Número de clusters")
plt.ylabel("WCSS")#suma de los cuadrados de las distancias
plt.show()

#%%
#APLICACION DEL MODELO
num_clusters = 9   #numero de cluster
model = KMeans(n_clusters = num_clusters, init = "k-means++", max_iter = 300, n_init = 10, random_state = 0)
y_clusters = model.fit_predict(x)
sns.countplot(y_clusters)

#SACAR LOSCENTROS DEL MODELO
centers = model.cluster_centers_

# FIGURA 3D PUNTOS GRANDES
# Diagrama de dispersión 3D usando plotly Nombre de los ejes
Scene = dict(xaxis = dict(title  = 'Entropía (bits)'),yaxis = dict(title  = 'Frecuencia fundamental (Hz)'),zaxis = dict(title  = 'Temperatura (°C)'),bgcolor='rgba(0,0,0,0)')
#nombre de los ejes
labels = model.labels_

# arreglo de colores
colors = ['yellow','blue','red','green','orange','cyan','pink','gray','brown','purple'] + ['yellow','blue','red','green','orange','cyan','pink','gray','brown','purple']
colors_cluster = ['color']*len(labels)
for i in range(num_clusters):
    for j in range(len(labels)):
        if(labels[j] == i):
            colors_cluster[j] = colors[i]
# Aqui se cambian los ejes, 0 casos,1 pp , 2 temp // también se deben cambiar los centros
trace = go.Scatter3d(x=x[:, 1], y=x[:, 0], z=x[:, 2], name='Clusters', mode='markers',marker=dict(color = colors_cluster, size= 10, line=dict(color= 'black',width = 10)))
trace2 = go.Scatter3d(x=centers[:, 1], y=centers[:, 0], z=centers[:, 2], name='Centers', mode='markers',marker=dict(color = 'black', size= 12, line=dict(color= 'black',width = 10)))
layout = go.Layout(margin=dict(l=0,r=0),scene = Scene,height = 800,width = 800,paper_bgcolor='white',)
data = [trace,trace2]
fig = go.Figure(data = data, layout = layout)
fig.show()

import matplotlib.pyplot as plt

legend_colors = [colors[i] for i in range(num_clusters)]

plt.figure(figsize=(2, 5))
for i, color in enumerate(legend_colors):
    plt.scatter(0.1, i * 0.2, c=color, s=100, marker='o', label=f'Clúster {i + 1}')

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.axis('off')
plt.savefig("legend.png", bbox_inches='tight')

# Guardar la imagen de la leyenda
plt.close()

#clusters
df_cluster = pd.DataFrame()
for i in range(num_clusters):
    for j in range(len(labels)):
        if(labels[j] == i):
            df_cluster = df_cluster.append(df.iloc[j])

    #df = pd.read_csv(r"bdbirdelevation5.csv")
    df_cluster.to_csv(r"Cluster_"+str(i)+".csv", index=False)
    df_cluster = pd.DataFrame()

In [ ]:
#%%
#DENDOGRAMA 2D
# #%%
import numpy as np
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram

from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import load_iris

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
# Load your data here
X = df.iloc[:, [1,5,8]].values

# Create AgglomerativeClustering model
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
model = model.fit(X)

plt.title("Dendrograma de agrupamiento jerárquico")
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode="level", p=3)
plt.xlabel("Número de puntos en el nodo.")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import pandas as pd

# Cargar los datos desde el archivo CSV (asegúrate de que tus datos estén en el archivo CSV)
data = pd.read_csv('Cluster_3.csv')

# Selecciona una columna específica para el análisis
columna_de_interes = 'entropy'  # Reemplaza 'NombreDeLaColumna' con el nombre de la columna que deseas analizar

# Extrae los datos de esa columna
data_columna = data[columna_de_interes]

# Lista de distribuciones para probar
distribuciones = [
    stats.norm,  # Distribución normal
    stats.expon,  # Distribución exponencial
    stats.lognorm,  # Distribución log-normal
    stats.gamma,  # Distribución gamma
    #stats.poisson,
    #stats.bernoulli,
    #stats.binom,
    stats.cauchy,
    stats.chi2,
    stats.t,
    stats.laplace,
    stats.pareto,
    stats.beta,
    # Agrega más distribuciones si es necesario
]

# Inicializa variables para almacenar los resultados
best_fit = None
best_aic = np.inf
best_params = None

# Ajusta cada distribución a los datos y calcula el AIC
for distribution in distribuciones:
    params = distribution.fit(data_columna)
    log_likelihood = distribution.logpdf(data_columna, *params).sum()
    aic = -2 * log_likelihood + 2 * len(params)
    if aic < best_aic:
        best_fit = distribution
        best_params = params
        best_aic = aic

# Crea un gráfico de probabilidad acumulativa (CDF) de los datos
sns.ecdfplot(data_columna, marker=".", label="Datos")

# Dibuja la mejor distribución ajustada en términos de CDF
x = np.linspace(min(data_columna), max(data_columna), 1000)
cdf = best_fit.cdf(x, *best_params)
plt.plot(x, cdf, label=f'{best_fit.name} mejor ajuste')

# Agrega etiquetas para mostrar los valores de los parámetros de las distribuciones
param_labels = ''

if best_fit.name == 'beta':
    param_labels = f'alpha = {best_params[0]:.2f}\nbeta = {best_params[1]:.2f}'
elif best_fit.name == 'gamma':
    param_labels = f'shape = {best_params[0]:.2f}\nscale = {best_params[1]:.2f}'
elif best_fit.name == 'lognorm':
    param_labels = f'sigma = {best_params[0]:.2f}\nloc = {best_params[1]:.2f}\nscale = {best_params[2]:.2f}'
elif best_fit.name == 'chi2':
    param_labels = f'dof = {best_params[0]:.2f}\nloc = {best_params[1]:.2f}\nscale = {best_params[2]:.2f}'
elif best_fit.name == 'norm':
    param_labels = f'loc = {best_params[0]:.2f}\nscale = {best_params[1]:.2f}'

plt.text(0.7, 0.2, param_labels, transform=plt.gca().transAxes, fontsize=12, va='top')



plt.title("Gráfico de Probabilidad Acumulativa (CDF)")
plt.legend()
plt.xlabel(columna_de_interes)
plt.ylabel("Probabilidad")
# Aquí quitamos la cuadrícula
plt.grid(False)
plt.show()